In [1]:
import polars as pl
import duckdb

Vamos testar a leitura usando duas bibliotecas que tem sido mais eficientes do que Pandas em benchmarks recentes: DuckDB, um banco de dados analítico "in-process", e Polars, um novo DataFrame escrito em Rust.

Vamos começar por DuckDB. Como é "in-process", ele carrega os arquivos em memória; para evitar esse comportamento e permitir "larger-than-memory workloads" (`https://duckdb.org/docs/guides/performance/how_to_tune_workloads`), vamos estabeler um `temp_directory`: assim os processos que excederem a memória do sistema são distribuídos pro disco.

In [2]:
con = duckdb.connect(config = {"temp_directory": "./temp_dir.tmp/"})

1. Benchmarks de tempo

In [3]:
def read_duckdb():
    df = con.sql("SELECT * FROM "./vendas.csv"")
    return df

def read_polars_all():
    df = pl.read_csv("./vendas.csv")
    return df

def read_polars_batches():
    df = pl.read_csv_batched("./vendas.csv")
    return df

def read_polars_lazy():
    df = pl.scan_csv("./vendas.csv")
    return df

In [12]:
%%timeit -r5 -n10
read_duckdb()

58.2 ms ± 1.2 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)


In [14]:
%%timeit -r5 -n10
read_polars_all()

473 ms ± 38.9 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)


In [15]:
%%timeit -r5 -n10
read_polars_batches()

339 μs ± 108 μs per loop (mean ± std. dev. of 5 runs, 10 loops each)


In [16]:
%%timeit -r5 -n10
read_polars_lazy()

The slowest run took 19.99 times longer than the fastest. This could mean that an intermediate result is being cached.
95.8 μs ± 143 μs per loop (mean ± std. dev. of 5 runs, 10 loops each)
